<a href="https://colab.research.google.com/github/Nikkuchan/CpE-AIML/blob/main/58090_LabNo06_Wagler/58090_LabNo06_Wagler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic 05.2: Perceptrons, Gradient Descent, and Backpropagation
$_{\text{©D.J. Lopez | 2021 | Fundamentals of Machine Learning}}$



## Laboratory Activity
1. For the laboratory activity, obtain a dataset of your liking from a data source. Explain the purpose of the dataset and mention any publication if it is obtained from the source. Provide a needs statement and significance for the dataset.

2. Identify an algorithm or method in performing a single or multiple variable classification using the Perceptron alogrithm. 

3. You must re-create your Perceptron algorithm with Gradient Descent and Backpropagation using your own code in a separate Google Colab. However, you are required to observe the following:

>* Enforce object-oriented programming by implementing at least two of the pillars of OOP in the entirety of the solution.
* Dedicated functions for training, predicting, and evaluating the solution.
* A DataFrame of the metrics of the solution
* A visualization of the solution’s results.


NOTES: https://github.com/dyjdlopez/fund-of-aiml/blob/main/activities/05%20-%20Classification/fund_aiml_05v1_lec2_2021.ipynb 


In [65]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import os

In [66]:
heartDisease = pd.read_csv("/content/heart_disease.csv")
y = heartDisease.target.values
x_data = heartDisease.drop(['target'], axis = 1)

In [67]:
x = (x_data - np.min(x_data)) / (np.max(x_data) - np.min(x_data)).values
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2,random_state=0)
x_train = x_train.T
y_train = y_train.T
x_test = x_test.T
y_test = y_test.T

In [80]:
class Perceptrons:
  def __init__(self, learningRate=1, iteration=100):
    self.learningRate = learningRate
    self.iteration = iteration
 
  def initialize(dimension):  
    weight = np.full((dimension,1),0.01)
    bias = 0.0
    return weight,bias
  
  def sigmoid(z):
    y_head = 1/(1+ np.exp(-z))
    return y_head

  def forwardBackward(weight,bias,x_train,y_train):
    # Forward Prp
    y_head = sigmoid(np.dot(weight.T,x_train) + bias)
    loss = -(y_train*np.log(y_head) + (1-y_train)*np.log(1-y_head))
    cost = np.sum(loss) / x_train.shape[1]   
    # Backward Prp
    derivative_weight = np.dot(x_train,((y_head-y_train).T))/x_train.shape[1]
    derivative_bias = np.sum(y_head-y_train)/x_train.shape[1]
    gradients = {"Derivative Weight" : derivative_weight, "Derivative Bias" : derivative_bias}
    return cost,gradients

  def train(weight,bias,x_train,y_train) :
    costList = []
    index = []
    for i in range(iteration):
        cost,gradients = forwardBackward(weight,bias,x_train,y_train)
        weight = weight - learningRate * gradients["Derivative Weight"]
        bias = bias - learningRate * gradients["Derivative Bias"]      
        costList.append(cost)
        index.append(i)
    parameters = {"weight": weight,"bias": bias}
    print("iteration:",iteration)
    print("cost:",cost)
    plt.plot(index,costList)
    plt.xlabel("Number of Iteration")
    plt.ylabel("Cost")
    plt.show()
    return parameters, gradients
  
  def predict(weight,bias,x_test):
    z = np.dot(weight.T,x_test) + bias
    y_head = sigmoid(z)

    y_prediction = np.zeros((1,x_test.shape[1]))
    
    for i in range(y_head.shape[1]):
        if y_head[0,i] <= 0.5:
            y_prediction[0,i] = 0
        else:
            y_prediction[0,i] = 1
    return y_prediction

  def logistic_regression(x_train,y_train,x_test,y_test):
    dimension = x_train.shape[0]
    weight,bias = initialize(dimension)
    parameters, gradients = train(weight,bias,x_train,y_train)
    y_prediction = predict(parameters["weight"],parameters["bias"],x_test)
    print("Test Accuracy: {:.2f}%".format((100 - np.mean(np.abs(y_prediction - y_test))*100)))

In [81]:
classification = Perceptrons()